In [1]:
import os
os.chdir("../")

In [6]:
from s3ts.data.stsdataset import STSDataset, LSTSDataset, StreamingTimeSeries
from s3ts.api.nets.encoders.dtw.dtw_layer import DTWLayer
from pytorch_lightning import Trainer

In [5]:
from s3ts.api.ucr import load_ucr_classification
from s3ts.api.ts2sts import fin_random_STS
import numpy as np
import torch

X, Y, mapping = load_ucr_classification("BasicMotions")
#X, Y, mapping = load_ucr_classification("GunPoint")
print(X.shape, Y.shape, len(np.unique(Y)))

STS, SCS = fin_random_STS(X, Y, length=60)
print(STS.shape, SCS.shape)

Loading 'BasicMotions' from cache...
(80, 6, 100) (80,) 4
(6, 6000) (6000,)


In [7]:
ds = StreamingTimeSeries(STS, SCS, wsize=32, wstride=1)

In [15]:
indices_shuffled = np.arange(ds.indices.shape[0])[:5000]
np.random.shuffle(indices_shuffled)

data_split = {
    "train": lambda x: np.isin(x, indices_shuffled),
    "val": lambda x: np.isin(x, np.arange(ds.indices.shape[0])[5064:5500]),
    "test": lambda x: np.isin(x, np.arange(ds.indices.shape[0])[5064:]),
}

dm = LSTSDataset(ds, data_split=data_split, batch_size=32, random_seed=42, num_workers=8)
dm.l_patterns = 16

In [16]:
from s3ts.api.nets.methods import create_model_from_DM, train_model

In [17]:
model = create_model_from_DM(dm, "dtw", "cnn", "mlp", "cls", "test", 8, 16, 1, 0.001)

Input shape:  torch.Size([1, 8, 16, 16])
Latent shape:  torch.Size([1, 16, 1, 16])
[tensor(256), 16, 4]


In [18]:
train_model(model=model, dm=dm, max_epochs=5)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name        | Type                 | Params
------------------------------------------------------
0  | dtw_layer   | DTWLayer             | 768   
1  | encoder     | CNN_IMG              | 4.1 K 
2  | decoder     | MultiLayerPerceptron | 4.2 K 
3  | flatten     | Flatten              | 0     
4  | softmax     | Softmax              | 0     
5  | train_acc   | MulticlassAccuracy   | 0     
6  | train_f1    | MulticlassF1Score    | 0     
7  | train_auroc | MulticlassAUROC      | 0     
8  | val_acc     | MulticlassAccuracy   | 0     
9  | val_f1      | MulticlassF1Score    | 0     
10 | val_auroc   | MulticlassAUROC      | 0     
11 | test_acc    | MulticlassAccuracy   | 0     
12 | test_f1     | MulticlassF1Score    | 0     
13 | test_auroc  | MulticlassAUROC      | 0     
----------------------------------------

Epoch 4: 100%|██████████| 156/156 [00:04<00:00, 38.08it/s, v_num=1, train_loss_step=0.992, val_loss=0.899, val_acc=0.557, val_auroc=0.0908, train_loss_epoch=0.869, train_acc=0.648, train_auroc=0.872]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 156/156 [00:04<00:00, 38.03it/s, v_num=1, train_loss_step=0.992, val_loss=0.899, val_acc=0.557, val_auroc=0.0908, train_loss_epoch=0.869, train_acc=0.648, train_auroc=0.872]
Input shape:  torch.Size([1, 8, 16, 16])
Latent shape:  torch.Size([1, 16, 1, 16])
[tensor(256), 16, 4]
Validation DataLoader 0: 100%|██████████| 14/14 [00:00<00:00, 67.20it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.5573394298553467     │
│         val_auroc         │    0.09083783626556396    │
│          val_f1           │    0.5573394298553467     │
│         val_loss          │    0.8994016051292419     │
└───────────────────────────┴───────────────────────────┘

(WrapperModel(
   (dtw_layer): DTWLayer()
   (encoder): CNN_IMG(
     (cnn_0): Sequential(
       (0): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=same)
       (1): ReLU()
       (2): MaxPool2d(kernel_size=(4, 1), stride=(4, 1), padding=0, dilation=1, ceil_mode=False)
     )
     (cnn_1): Sequential(
       (0): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
       (1): ReLU()
       (2): AvgPool2d(kernel_size=(4, 1), stride=(4, 1), padding=0)
       (3): Dropout(p=0.35, inplace=False)
     )
     (cnn_2): Sequential(
       (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
       (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): Dropout(p=0.4, inplace=False)
     )
     (cnn_3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
   )
   (decoder): MultiLayerPerceptron(
     (fcn_layer_0): Linear(in_features=256, out_features=16, bias=True)
     (act_layer_0): ReL